# Part3 実践例 Chap.8 回帰問題のコンペ

## 8.1 MLB Player Digital Engagement Forecasting コンペの概要

## 8.2 分析のステップ

- 1 ベースライン作成
  - モデル: LightGBM
  - バリデーション設計: 時系列を加味したtrain/validの分割
  - 評価指数: MCMAE
- 2 特徴量エンジニアリング
  - ほかのカラム・テーブルも活用して特徴量を生成
  - ラグ特徴量
- 3 モデルチューニング
  - ニューラルネットワークの適用


- MCMAE: Mean Columns-wise Mean Absolute Error)
- MAE: Mean Absolute Error

## 8.3 ベースライン作成

### 8.3.1 分析設計

### 8.3.2 データ前処理

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import gc
import pickle
import os
import datetime as dt

# plot
import matplotlib.pyplot as plt

# LightGBM
import lightgbm as lgb

from sklearn.metrics import mean_absolute_error

import warnings

warnings.simplefilter("ignore")

# 表示桁数の指定
pd.options.display.float_format = "{:10.4f}".format
pd.set_option("display.max_columns", None)

#### train_updated.csvの読み込みと加工

In [2]:
# train_updated.csvファイルの読み込み
train = pd.read_csv("./mlb-player-digital-engagement-forecasting/train_updated.csv")
print(train.shape)
train.head()

(1308, 12)


,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [3]:
# 処理速度を上げるためにデータを絞り込む
# train: 2020-04~2021-03
# test: 2021-04
train = train.loc[train["date"] >= 20200401, :].reset_index(drop=True)
print(train.shape)

(487, 12)


In [4]:
# json形式のカラムを表形式に変換する
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)


def extract_data(input_df, col="events", show=False):
    output_df = pd.DataFrame()
    for i in np.arange(len(input_df)):
        if show:
            print("\r{}/{}".format(i + 1, len(input_df)), end="")
        try:
            output_df = pd.concat([output_df, unpack_json(input_df[col].iloc[i])], axis=0, ignore_index=True)
        except:
            pass
    if show:
        print("")
    if show:
        print(output_df.shape)
    if show:
        display(output_df.head())
    return output_df

In [5]:
# train_updated.csvから「nextDayPlayerEngagement」を取り出して表形式に変換
df_engagement = extract_data(train, col="nextDayPlayerEngagement", show=True)

487/487
(1003707, 6)


,engagementMetricsDate,playerId,target1,target2,target3,target4
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967


In [6]:
# 結合キーであるdate_playerIdの作成
df_engagement["date_playerId"] = (
    df_engagement["engagementMetricsDate"].str.replace("-", "") + "_" + df_engagement["playerId"].astype(str)
)
df_engagement.head()

,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947,20200402_425794
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304,20200402_571704
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850,20200402_506702
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900,20200402_607231
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967,20200402_543193


In [7]:
# 推論実施日のカラム作成（推論実施日＝推論対象日の前日）
df_engagement["date"] = pd.to_datetime(df_engagement["engagementMetricsDate"], format="%Y-%m-%d") + dt.timedelta(
    days=-1
)

# 推論実施日から「曜日」と「年月」の特徴量作成
df_engagement["dayofweek"] = df_engagement["date"].dt.dayofweek
df_engagement["yearmonth"] = df_engagement["date"].astype(str).apply(lambda x: x[:7])
df_engagement.head()

,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId,date,dayofweek,yearmonth
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947,20200402_425794,2020-04-01,2,2020-04
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304,20200402_571704,2020-04-01,2,2020-04
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850,20200402_506702,2020-04-01,2,2020-04
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900,20200402_607231,2020-04-01,2,2020-04
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967,20200402_543193,2020-04-01,2,2020-04


#### player.csvの読み込みと加工

In [24]:
# player.csvの読み込みと加工
df_players = pd.read_csv("./mlb-player-digital-engagement-forecasting/players.csv")
print(df_players.shape)
print(df_players["playerId"].aggregate("nunique"))
df_players.head()

(2061, 12)
2061


,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True


In [26]:
# 評価対象の人数確認
# True == 1
# df_players["playerForTestSetAndFuturePreds"] = np.where(df_players["playerForTestSetAndFuturePreds"] == True, 1, 0)
df_players["playerForTestSetAndFuturePreds"] = np.where(df_players["playerForTestSetAndFuturePreds"] == 1, 1, 0)
print(df_players["playerForTestSetAndFuturePreds"].sum())
print(df_players["playerForTestSetAndFuturePreds"].mean())

1187
0.5759340126152354


### 8.3.3 データセット作成

In [28]:
# テーブルの結合
df_train = pd.merge(df_engagement, df_players, on=["playerId"], how="left")
print(df_train.shape)
display(df_train.head())

(1003707, 21)


,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId,date,dayofweek,yearmonth,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947,20200402_425794,2020-04-01,2,2020-04,Adam Wainwright,1981-08-30,2005-09-11,Brunswick,GA,USA,79,230,1,Pitcher,1
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304,20200402_571704,2020-04-01,2,2020-04,Ken Giles,1990-09-20,2014-06-12,Albuquerque,NM,USA,75,210,1,Pitcher,0
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850,20200402_506702,2020-04-01,2,2020-04,Sandy Leon,1989-03-13,2012-05-14,Maracaibo,NaN,Venezuela,70,235,2,Catcher,1
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900,20200402_607231,2020-04-01,2,2020-04,John Gant,1992-08-06,2016-04-06,Savannah,GA,USA,76,200,1,Pitcher,1
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967,20200402_543193,2020-04-01,2,2020-04,Drew Gagnon,1990-06-26,2018-07-10,Columbia,CA,USA,76,215,1,Pitcher,0


In [29]:
# 学習用データセットの作成
x_train = df_train[
    [
        "playerId",
        "dayofweek",
        "birthCity",
        "birthStateProvince",
        "birthCountry",
        "heightInches",
        "weight",
        "primaryPositionCode",
        "primaryPositionName",
        "playerForTestSetAndFuturePreds",
    ]
]
y_train = df_train[["target1", "target2", "target3", "target4"]]
id_train = df_train[
    ["engagementMetricsDate", "playerId", "date_playerId", "date", "yearmonth", "playerForTestSetAndFuturePreds"]
]
print(x_train.shape, y_train.shape, id_train.shape)
x_train.head()

(1003707, 10) (1003707, 4) (1003707, 6)


,playerId,dayofweek,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,425794,2,Brunswick,GA,USA,79,230,1,Pitcher,1
1,571704,2,Albuquerque,NM,USA,75,210,1,Pitcher,0
2,506702,2,Maracaibo,NaN,Venezuela,70,235,2,Catcher,1
3,607231,2,Savannah,GA,USA,76,200,1,Pitcher,1
4,543193,2,Columbia,CA,USA,76,215,1,Pitcher,0


In [30]:
# カテゴリ変数をcategory方に変換
for col in [
    "playerId",
    "dayofweek",
    "birthCity",
    "birthStateProvince",
    "birthCountry",
    "primaryPositionCode",
    "primaryPositionName",
]:
    x_train[col] = x_train[col].astype("category")

print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003707 entries, 0 to 1003706
Data columns (total 10 columns):
 #   Column                          Non-Null Count    Dtype   
---  ------                          --------------    -----   
 0   playerId                        1003707 non-null  category
 1   dayofweek                       1003707 non-null  category
 2   birthCity                       1003707 non-null  category
 3   birthStateProvince              738292 non-null   category
 4   birthCountry                    1003707 non-null  category
 5   heightInches                    1003707 non-null  int64   
 6   weight                          1003707 non-null  int64   
 7   primaryPositionCode             1003707 non-null  category
 8   primaryPositionName             1003707 non-null  category
 9   playerForTestSetAndFuturePreds  1003707 non-null  int64   
dtypes: category(7), int64(3)
memory usage: 31.7 MB
None


### 8.3.4 バリデーション設計